# Pokémon TCG Synergy Engine
---

In [1]:
# import libraries for data wrangling
import ast
import re
import pandas as pd
import numpy as np
import src

## Pokémon TCG Data Wrangling
---

In [2]:
data = src.get_card_data()
df_pokemon_cards = src.prep_card_data(data)

In [3]:
trainer = pd.read_csv('data/cards.csv')
trainer = trainer[trainer['supertype']=='Trainer']

In [4]:
trainer['name'] = trainer['name'].str.replace(r'\s*\(.*?\)', '', regex=True).str.strip()

In [5]:
trainer = trainer.sort_values(by=['name', 'subtypes'],ascending=False)
trainer = trainer[(trainer['regulationMark']>='G')].reset_index(drop=True)
trainer = trainer.drop_duplicates(subset=['name'],
                                  keep='first').reset_index(drop=True)

In [6]:
trainer['subtypes'] = trainer['subtypes'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else None)

In [7]:
trainer.dropna(subset=['subtypes']).reset_index(inplace=True)

In [8]:
def extract_subtype(subtype):
    '''Extract the Stage of each Pokémon card'''
    if not isinstance(subtype, list):
        return None
    else:
        return subtype[0]

# Apply the function to your DataFrame
trainer['subtypes_extracted'] = trainer['subtypes'].apply(extract_subtype)

In [9]:
trainer = trainer.drop(columns=['artist', 'ancientTrait', 'cardmarket', 'flavorText',
                                'images', 'nationalPokedexNumbers', 'rarity', 'retreatCost',
                                'tcgplayer', 'resistances', 'weaknesses'])

In [10]:
trainer['is_ace_spec'] = trainer['subtypes'].apply(lambda x: 'ACE SPEC' in x)
trainer['is_future'] = trainer['subtypes'].apply(lambda x: 1 if 'Future' in x else 0)
trainer['is_ancient'] = trainer['subtypes'].apply(lambda x: 1 if 'Ancient' in x else 0)

In [11]:
dataset=trainer.copy()

In [12]:
for col in ['abilities', 'attacks', 'set', 'rules']:
    dataset[col] = dataset[col].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else x)

In [13]:
dataset['set_name'] = dataset['set'].apply(lambda x: x.get('id') if isinstance(x, dict) else None)
dataset.rename(columns={'number': 'set_number'}, inplace=True)
dataset['set_number'] = dataset['set_number'].astype(int)

In [14]:
dataset.sort_values(by=['set_name','set_number'])

,abilities,attacks,convertedRetreatCost,evolvesFrom,hp,id,legalities,regulationMark,name,set_number,rules,set,subtypes,supertype,types,subtypes_extracted,is_ace_spec,is_future,is_ancient,set_name
231,NaN,NaN,NaN,NaN,NaN,sv1-166,"{'unlimited': 'Legal', 'expanded': 'Legal', 's...",G,Arven,166,[Search your deck for an Item card and a Pokém...,"{'id': 'sv1', 'images': {'symbol': 'https://im...",[Supporter],Trainer,NaN,Supporter,False,0,0,sv1
226,NaN,NaN,NaN,NaN,NaN,sv1-167,"{'unlimited': 'Legal', 'expanded': 'Legal', 's...",G,Beach Court,167,[The Retreat Cost of each Basic Pokémon in pla...,"{'id': 'sv1', 'images': {'symbol': 'https://im...",[Stadium],Trainer,NaN,Stadium,False,0,0,sv1
191,NaN,NaN,NaN,NaN,NaN,sv1-168,"{'unlimited': 'Legal', 'expanded': 'Legal', 's...",G,Crushing Hammer,168,"[Flip a coin. If heads, discard an Energy from...","{'id': 'sv1', 'images': {'symbol': 'https://im...",[Item],Trainer,NaN,Item,False,0,0,sv1
183,NaN,NaN,NaN,NaN,NaN,sv1-169,"{'unlimited': 'Legal', 'expanded': 'Legal', 's...",G,Defiance Band,169,[If you have more Prize cards remaining than y...,"{'id': 'sv1', 'images': {'symbol': 'https://im...",[Pokémon Tool],Trainer,NaN,Pokémon Tool,False,0,0,sv1
173,NaN,NaN,NaN,NaN,NaN,sv1-170,"{'unlimited': 'Legal', 'expanded': 'Legal', 's...",G,Electric Generator,170,[Look at the top 5 cards of your deck and atta...,"{'id': 'sv1', 'images': {'symbol': 'https://im...",[Item],Trainer,NaN,Item,False,0,0,sv1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,NaN,NaN,NaN,NaN,NaN,sv9-156,"{'unlimited': 'Legal', 'expanded': 'Legal', 's...",I,Redeemable Ticket,156,"[Count your Prize cards, shuffle them, and put...","{'id': 'sv9', 'images': {'symbol': 'https://im...",[Item],Trainer,NaN,Item,False,0,0,sv9
48,NaN,NaN,NaN,NaN,NaN,sv9-157,"{'unlimited': 'Legal', 'expanded': 'Legal', 's...",I,Ruffian,157,[Discard a Pokémon Tool and a Special Energy f...,"{'id': 'sv9', 'images': {'symbol': 'https://im...",[Supporter],Trainer,NaN,Supporter,False,0,0,sv9
36,NaN,NaN,NaN,NaN,NaN,sv9-158,"{'unlimited': 'Legal', 'expanded': 'Legal', 's...",I,Super Potion,158,[Heal 60 damage from 1 of your Pokémon. If you...,"{'id': 'sv9', 'images': {'symbol': 'https://im...",[Item],Trainer,NaN,Item,False,0,0,sv9
85,NaN,NaN,NaN,NaN,NaN,svp-45,"{'unlimited': 'Legal', 'expanded': 'Legal', 's...",G,Paradise Resort,45,[The Retreat Cost of each Psyduck in play (bot...,"{'id': 'svp', 'images': {'symbol': 'https://im...",[Stadium],Trainer,NaN,Stadium,False,0,0,svp


In [17]:
dataset['subtypes_extracted'].value_counts()

subtypes_extracted
Supporter       87
Item            81
Pokémon Tool    47
Stadium         29
Name: count, dtype: int64

In [19]:
items = dataset[dataset['subtypes_extracted']=='Item']